In [1]:
import os
import random
import pandas as pd
from IPython.display import display, HTML, clear_output
from sklearn.metrics import cohen_kappa_score

# --- KONFIGURATION ---
ORIGINAL_CSV = "BEWERTUNGSBOGEN_FINAL.csv"
IMG_DIR = "./blind_study" 
NUM_GROUPS_TO_TEST = 20  # 20 Blätter * 4 Modelle = 80 Bilder

def run_reliability_test_80():
    # 1. Daten laden
    if not os.path.exists(ORIGINAL_CSV):
        print(f"FEHLER: Datei {ORIGINAL_CSV} nicht gefunden!")
        return

    df = pd.read_csv(ORIGINAL_CSV)
    rating_col = "Rating (1=Noise, 3=Mix, 5=Precise)"
    
    # Rating bereinigen
    df[rating_col] = pd.to_numeric(df[rating_col], errors='coerce')
    
    # Nur bereits bewertete Bilder nehmen (wir brauchen ja den Vergleichswert)
    df_rated = df.dropna(subset=[rating_col])
    
    # Image_IDs holen (Gruppen)
    if "Image_ID" in df_rated.columns:
        all_ids = df_rated["Image_ID"].unique().tolist()
    else:
        # Fallback: Falls Image_ID fehlt, versuchen wir sie aus dem Filename zu raten oder brechen ab
        print("FEHLER: Spalte 'Image_ID' fehlt in der CSV. Bitte führe erst den Reparatur-Schritt aus!")
        return
    
    # Check ob genug Daten da sind
    if len(all_ids) < NUM_GROUPS_TO_TEST:
        print(f"ACHTUNG: Du hast erst {len(all_ids)} Blätter bewertet.")
        print(f"Kann keine {NUM_GROUPS_TO_TEST} für den Test auswählen.")
        print("Bitte bewerte erst mehr Bilder im Haupt-Skript!")
        return
        
    # 2. ZUFALLSAUSWAHL
    test_ids = random.sample(all_ids, NUM_GROUPS_TO_TEST)
    
    print(f"Start: Reliabilitäts-Check für {NUM_GROUPS_TO_TEST} Blätter (= 80 Bilder).")
    print("Modus: Blind (Alte Note ist versteckt).")
    print("-" * 50)
    
    results = []
    
    # 3. DER LOOP (20 Durchgänge)
    for i, img_id in enumerate(test_ids):
        # Alle 4 Modelle zu diesem Bild holen
        group = df[df["Image_ID"] == img_id]
        # HTML Anzeige bauen (Side-by-Side)
        html_str = "<div style='display: flex; flex-direction: row; gap: 10px; justify-content: center;'>"
        current_batch = []
        
        for pos_idx, (idx, row) in enumerate(group.iterrows()):
            filename = row["Rating_ID (Filename)"]
            old_rating = int(row[rating_col])
            true_class = row["True_Class"]
            img_path = os.path.join(IMG_DIR, filename)
            
            # Prüfen ob Bewertung schon da ist (falls man mittendrin abgebrochen hat)
            current_rating = row[rating_col]
            rating_text = f"Aktuell: {int(current_rating)}" if pd.notna(current_rating) else "Noch offen"
            # Fallback für die Anzeige, falls Pfad Probleme macht:
            if os.path.exists(img_path):
                 html_str += f"""
                 <div style='text-align: center; width: 24%; border: 1px solid #eee; padding: 5px;'>
                    <h3 style='margin:0; font-size: 14px;'>Bild {len(current_batch)+1}</h3>
                    <small style='color: #555;'>Ziel: <b>{true_class}</b></small><br>
                    
                    <div style='height: 250px; display: flex; align-items: center; justify-content: center;'>
                        <img src='{img_path}' style='max-width: 100%; max-height: 100%; object-fit: contain;'>
                    </div>
                    
                    <small style='display:block; overflow:hidden; text-overflow:ellipsis; white-space:nowrap;'>{filename}</small>
                    <strong>{rating_text}</strong>
                 </div>
                 """
            else:
                 html_str += f"<div style='width: 24%;'>BILD FEHLT: {filename}</div>"
            
            current_batch.append({'filename': filename, 'old': old_rating})
            
        html_str += "</div>"
        
        # Anzeigen
        print(f"[{i+1}/{NUM_GROUPS_TO_TEST}] Re-Test Blatt ID: {img_id}")
        display(HTML(html_str))
        
        # Eingabe Loop (4x pro Blatt)
        for pos, item in enumerate(current_batch):
            valid = False
            while not valid:
                user_input = input(f"Rating für Bild {pos+1} (von links) [1-5]: ")
                if user_input in ['1', '2', '3', '4', '5']:
                    results.append({
                        'Filename': item['filename'],
                        'Old_Rating': item['old'],
                        'New_Rating': int(user_input)
                    })
                    valid = True
                else:
                    print("Ungültig. Bitte 1-5.")
        
        # Screen leeren für nächstes Blatt
        clear_output(wait=True)

    # 4. STATISTISCHE AUSWERTUNG
    print("\n" + "="*40)
    print("ERGEBNIS DEINES RELIABILITÄTS-TESTS")
    print("="*40)
    
    df_res = pd.DataFrame(results)
    
    # Metrik 1: Exakte Matches
    df_res['Exact_Match'] = df_res['Old_Rating'] == df_res['New_Rating']
    exact_acc = df_res['Exact_Match'].mean() * 100
    
    # Metrik 2: Toleranz (+/- 1)
    df_res['Diff'] = abs(df_res['Old_Rating'] - df_res['New_Rating'])
    tolerant_match = (df_res['Diff'] <= 1).mean() * 100
    
    # Metrik 3: Cohen's Kappa
    kappa_msg = ""

    # linear weights bestrafen kleine Abweichungen weniger als große
    k_score = cohen_kappa_score(df_res['Old_Rating'], df_res['New_Rating'], weights='linear')
    kappa_msg = f"{k_score:.3f}"
    
    # Interpretation Kappa
    if k_score > 0.8: k_text = "(Exzellent)"
    elif k_score > 0.6: k_text = "(Gut)"
    elif k_score > 0.4: k_text = "(Moderat)"
    else: k_text = "(Schwach)"


    print(f"\nAnzahl geprüfter Bilder: {len(df_res)}")
    print(f"\n1. Exakte Übereinstimmung:      {exact_acc:.1f}%  (Ziel: >60%)")
    print(f"2. Toleranz-Treffer (+/- 1):    {tolerant_match:.1f}%  (Ziel: >85%)")
    print(f"3. Cohen's Kappa (Weighted):    {kappa_msg} {k_text}")
    
    # Fehleranalyse anzeigen
    bad_fails = df_res[df_res['Diff'] > 1]
    if len(bad_fails) > 0:
        print(f"\nACHTUNG: Bei {len(bad_fails)} Bildern hast du deine Meinung stark geändert (>1 Note Unterschied):")
        print(bad_fails[['Filename', 'Old_Rating', 'New_Rating']].to_string(index=False))
    else:
        print("\nPerfekt! Keine groben Abweichungen (>1 Note).")

    # Speichern als CSV für den Anhang
    df_res.to_csv("reliability_test_results.csv", index=False)
    print("\nDetails gespeichert in 'reliability_test_results.csv'.")

# Starten
run_reliability_test_80()


ERGEBNIS DEINES RELIABILITÄTS-TESTS

Anzahl geprüfter Bilder: 80

1. Exakte Übereinstimmung:      88.8%  (Ziel: >60%)
2. Toleranz-Treffer (+/- 1):    100.0%  (Ziel: >85%)
3. Cohen's Kappa (Weighted):    0.900 (Exzellent)

Perfekt! Keine groben Abweichungen (>1 Note).

Details gespeichert in 'reliability_test_results.csv'.
